<a id='Homework'></a>
# Homework

Thoery (5 points):
- Complete theory questions in Google Form
- Take a look at all links 
- Read and analyze all theory `TODO`s. In this lecture they are highly important 

Practice (10-15 points):
- First Option (Easier) - Apply [Sentence Classification/Sentence Regression](#Sentence_Classification) approach to [CommonLit - Evaluate Student Summaries](https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries)
    - Max Base Points: 10
    - Tune `BERT` / `DEBERTA` or other bert-like models, maybe re-write it. Tune all other hyper-params.
    - Try different optimizers lr schedulers 
    - Implement Cross-Validation and add test houldout
    - Make a submit to Kaggle
- Second Option (Harder) - Apply [Token Classification](#Token_Classification) approach to Location Detection task
    - Max Base Points: 15
    - Introduce fixes, mentioned in `TODO`s
    - Your model also have to handle ukrainian and russian languages. Make sure to add `../data/mantis_analytics_location_detection/ru_geo_dataset.csv` (from https://www.kaggle.com/datasets/vladimirsydor/mantis-analytics-location-detection/data). Think about
        - New Validation
        - New Word Embeddings
        - Maybe separate models
        - IMPORTANT: Take into account `doc_id`
        - Take into account that markup is far from ideal. Maybe pre-processing may help
    - Coming back to Lecture 3. F1 is the final production metric but it hardly depends on threshold. Maybe you can use some "soft" metric for model comparison ? 
    - Do we need additional post-processing ?
    - Tune `BERT` / `DEBERTA` or other bert-like models on the whole dataset for more epochs, maybe re-write it. Tune all other hyper-params.
    - Try different optimizers lr schedulers 
    - Implement Cross-Validation and add test houldout
    - Make a submit to Kaggle
    - Additional points: first private score - 25 points, second private score - 15 points, third private score - 10 points
    - Do not hesitate to use `Discussion` and `Code` on Kaggle. All additional useful insights will be also granted with additional scores
        - If you do not want to share with other competitors - you can share with lectors. It can be useful for Mantis usecase 
- Third Option (Hardest). Do both first and second options
- Form will contain separate places for First and Second Options. Just put `-` for Option, which you have not choosen 

<a id='homework_first_option'></a>
## First Option

## Imports

In [1]:

!pip install wandb
import wandb

import re
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["TOKENIZERS_PARALLELISM"]="true"
os.environ["WANDB_API_KEY"] = ''
os.environ["WANDB_PROJECT"]="nlp_course_deberta_finetuning"
os.environ["WANDB_LOG_MODEL"]="end" # upload best model to wandb artifacts after training
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from transformers import (AutoTokenizer, DataCollatorWithPadding, AutoConfig,
                          AutoModelForSequenceClassification, TrainingArguments, Trainer,
                          AdamW, get_constant_schedule_with_warmup)
from transformers.optimization import Adafactor, AdafactorSchedule
from datasets import Dataset

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 5.4 MB/s eta 0:00:00
     -------------------------------------- 190.6/190.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 241.0/241.0 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8801 sha256=71fd12a06d1f4ec22f4787bbf262995e1015ad86cf427fb63382d4aff9600373
  Stored in directory: c:\users\yevhe\appdata\local\pip\cache\wheels\20\7c\09\4ad42725a29fce4bc21137c7f25f062b3655a4aea5b0e8d9a2
Successfully built pathtools


'cpu'

In [2]:
n_samples_for_train = 10# 7165  # down-sample for quick demo
#  # 7165 - full size for CommonLit challenge
# 10 samples - 73 seconds for the full notebook run on CPU, the main thing - training part it takes 66 seconds
# 20 samples - 2.4 minutes passed total 
# 100 samples - 564 seconds (CPU) - full , and 8min 53s - for training the model
# 100 samples - 41.5 seconds (CPU) - full , and 16.5 s- for training the model
# 1000 samples - GPU - 2.1 minutes passed total 
# full - 7165 samples - GPU -  12.1 minutes passed total 
# full - 7165 samples - CPU -  7h.21min = 441 minutes passed total -- extemely slow


model_name="debertav3base"

class CFG:
    model_name=model_name #"debertav3base"
    model_path=os.path.join('/kaggle', 'input', model_name)
#     initialized_params_learning_rate=1e-6
#     non_initialized_params_learning_rate=1e-4
    optimizer='adafactor'
    use_fp16=True
    learning_rate=None # 2e-5
    weight_decay=0.03
    hidden_dropout_prob=0 # 0.005
    attention_probs_dropout_prob=0.007 # 0.005
    num_train_epochs=6
    n_splits=4
    batch_size=12
    random_seed=42
    save_steps=500
    max_length=512
    warmup_steps=100
    predict_mode=False
    report_to='wandb' if not predict_mode else 'none'
    run_name=f'{model_name}_{optimizer}_lr{learning_rate}_dropout{hidden_dropout_prob}' if report_to=='wandb' else None
    if predict_mode:
        pretrained_model_path = '../input/model-debertav3base-adam-lr1e-05-dropout5e-3-fold3/model-debertav3base_adam_lr1e-05_dropout5e-3_fold3'
#         model_artifact = 'yevhen-herasimov/nlp_course_deberta_finetuning/model-debertav3base_adam_lr1e-05_dropout5e-3_fold3:latest'
        report_to = 'none'
        run_name = None

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
seed_everything(CFG.random_seed)

## Load data

In [ ]:
DATA_FOLDER = os.path.join('..', 'input')
COMMONLIT_DATA_FOLDER = os.path.join(DATA_FOLDER, 'commonlit-evaluate-student-summaries')
COMMONLIT_TRAIN_SUMMARIES_PATH = os.path.join(COMMONLIT_DATA_FOLDER, 'summaries_train.csv')
COMMONLIT_TRAIN_PROMPTS_PATH = os.path.join(COMMONLIT_DATA_FOLDER, 'prompts_train.csv')
COMMONLIT_TEST_SUMMARIES_PATH = os.path.join(COMMONLIT_DATA_FOLDER, 'summaries_test.csv')
COMMONLIT_TEST_PROMPTS_PATH = os.path.join(COMMONLIT_DATA_FOLDER, 'prompts_test.csv')


In [ ]:
def load_data():
    
    train_df = pd.read_csv(COMMONLIT_TRAIN_SUMMARIES_PATH)
    train_pt_df = pd.read_csv(COMMONLIT_TRAIN_PROMPTS_PATH)
    test_df = pd.read_csv(COMMONLIT_TEST_SUMMARIES_PATH)
    test_pt_df = pd.read_csv(COMMONLIT_TEST_PROMPTS_PATH)
    
    train_df = pd.merge(train_df, train_pt_df, on='prompt_id')
    test_df = pd.merge(test_df, test_pt_df, on='prompt_id')
    
    return train_df, test_df


train_df, test_df = load_data()

In [ ]:
# print('Train data - not downsampled shape:', train_df.shape)

# train_df = train_df.groupby('prompt_id', group_keys=False).apply(lambda x: x.sample(20)) # downsampling for quick runs
# print('Train data - AFTER downsampling shape:', train_df.shape)
# display(train_df.head(10))

## EDA

In [ ]:
print('Plot targets:')
sns.scatterplot(x = train_df['content'], y = train_df['wording'], hue = train_df['prompt_id'] )
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Distribution of values in target columns')

axes[0].hist(train_df['content'], bins=100)
axes[0].set_title('Content column')
axes[1].hist(train_df['wording'], bins=100)
axes[1].set_title('Wording column')

plt.show();

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Boxplots of values in target columns')

axes[0].boxplot(train_df['content'])
axes[0].set_title('Content column')
axes[1].boxplot(train_df['wording'])
axes[1].set_title('Wording column')

plt.show();

In [ ]:
# Content column
q1 = np.percentile(train_df['content'], 25)
q3 = np.percentile(train_df['content'], 75)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
num_outliers_content = len(train_df['content'][train_df['content'] > upper_bound])

# Wording column
q1 = np.percentile(train_df['wording'], 25)
q3 = np.percentile(train_df['wording'], 75)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr
num_outliers_wording = len(train_df['wording'][train_df['wording'] > upper_bound])

# Print results
print("Number of data points outside Q3+1.5IQR in Content column:", num_outliers_content)
print("Number of data points outside Q3+1.5IQR in Wording column:", num_outliers_wording)

Number of statistical outliers is not high, but **we can test sensitive and non-sensitive to outliers loss functions**

**Check sequence length after preprocessing and tokenization**

In [ ]:
def preprocess(summary):
    # remove \n, \r, \t characters from text
    summary = re.sub(r'[\n\r\t]', ' ', summary)
    # remove extra whitespaces
    summary = re.sub(r'\s+', ' ', summary)
    return summary

tokenizer = AutoTokenizer.from_pretrained(os.path.join(CFG.model_path))

In [ ]:
train_df['clean_text'] = train_df['text'].apply(lambda summary: preprocess(summary))
tokenized_df = tokenizer(train_df['clean_text'].to_list())["input_ids"]

# Figure out number of tokens in each text
train_n_tokens = list(map(len, tokenized_df))

plt.title("Train N Tokens Distribution")
plt.hist(train_n_tokens, bins=100)
plt.show()

print(f'Number of texts above {CFG.max_length} tokens: {sum(n_tokens > CFG.max_length for n_tokens in train_n_tokens)}')

## Preprocessing

In [ ]:
# Inspired by https://www.kaggle.com/code/tsunotsuno/updated-debertav3-lgbm-with-spell-autocorrect
# Rewrote for own purposes
class DatasetPreprocessor:
    """
    Combines text preprocessing, tokenization, dataset creation into a single class
    """
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)
        
    def preprocess(summary):
        # remove \n, \r, \t characters from text
        summary = re.sub(r'[\n\r\t]', ' ', summary)
        # remove extra whitespaces
        summary = re.sub(r'\s+', ' ', summary)
        return summary

    
    def tokenize_function(self, sample):
        # Deberta isn't limited to 512 input length, because of relative position embeddings, 
        # but it will slow down significantly after 512 tokens
        tokenized = self.tokenizer(
            sample['clean_text'],
            padding='max_length',
            truncation=True,
            max_length=CFG.max_length)
        
        return {**tokenized, 'labels': [sample['content'], sample['wording']]}
      
        
    def tokenize_function_test(self, sample):
        # Deberta isn't limited to 512 input length, because of relative position embeddings, 
        # but it will slow down significantly after 512 tokens
        tokenized = self.tokenizer(
            sample['clean_text'],
            padding='max_length',
            truncation=True,
            max_length=CFG.max_length)
        
        return {**tokenized}
        
        
    def run(self, dataset, train_mode):
        dataset['clean_text'] = dataset['text'].apply(lambda summary: preprocess(summary))
        dataset = Dataset.from_pandas(dataset)
        if train_mode:
            tokenized_dataset = dataset.map(self.tokenize_function)
            tokenized_dataset = tokenized_dataset.remove_columns(['content', 'wording'])
        else:
            tokenized_dataset = dataset.map(self.tokenize_function_test)
            
        # delete common text columns
        tokenized_dataset = tokenized_dataset.remove_columns(['text', 'clean_text'])
            
        return tokenized_dataset

    
preprocessor = DatasetPreprocessor(model_name=CFG.model_name)

In [ ]:
def compute_rmse(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(targets, predictions):
    content_rmse = torch.sqrt(torch.mean((targets[:, 0] - predictions[:, 0]) ** 2))
    wording_rmse = torch.sqrt(torch.mean((targets[:, 1] - predictions[:, 1]) ** 2))
    mcrmse = torch.mean(torch.cat((content_rmse.reshape(-1), wording_rmse.reshape(-1)), dim=0))
    return mcrmse

def compute_mcrmse_numpy(eval_pred):
    predictions, targets = eval_pred
    content_rmse = np.sqrt(np.mean((targets[:, 0] - predictions[:, 0]) ** 2))
    wording_rmse = np.sqrt(np.mean((targets[:, 1] - predictions[:, 1]) ** 2))
    mcrmse = np.mean(np.concatenate((content_rmse.reshape(-1), wording_rmse.reshape(-1)), axis=0))
    return {"mcrmse": mcrmse}

## Modeling

In [ ]:
class DebertaRegressorModel:
    def __init__(self):
        self.data_collator = DataCollatorWithPadding(
            preprocessor.tokenizer
        )
        self.model_config = AutoConfig.from_pretrained(CFG.model_path)
        self.model_config.update({
            "hidden_dropout_prob": CFG.hidden_dropout_prob,
            "attention_probs_dropout_prob": CFG.attention_probs_dropout_prob,
            "num_labels": 2,
            "problem_type": "regression"
        })

    
    def train(self, fold_num, train_dataset, valid_dataset):
                
        model = AutoModelForSequenceClassification.from_pretrained(
            CFG.model_path,
            config=self.model_config
        )
        
        if CFG.optimizer == 'adam':
#             optimizer = AdamW([
#                 {'params': list(model.deberta.parameters()), 'lr': CFG.initialized_params_learning_rate},
#                 {'params': list(model.pooler.parameters()), 'lr': CFG.non_initialized_params_learning_rate},
#                 {'params': list(model.classifier.parameters()), 'lr': CFG.non_initialized_params_learning_rate}
#             ])
            optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)
            lr_scheduler = get_constant_schedule_with_warmup(
                optimizer,
                num_warmup_steps = CFG.warmup_steps
            )
        elif CFG.optimizer == 'adafactor':
            optimizer = Adafactor(
                model.parameters(), 
                scale_parameter=True, 
                relative_step=True, 
                warmup_init=True, 
                lr=CFG.learning_rate
            )
            lr_scheduler = AdafactorSchedule(optimizer)
         
        
        # TrainingArguments is used for all hyperparameters and configs
        training_args = TrainingArguments(
            output_dir = os.path.join(CFG.model_name, f'fold_{fold_num}'), # f'fold_{fold_num}'
            
            # model training arguments
            learning_rate=CFG.learning_rate,
            per_device_train_batch_size=CFG.batch_size,
            per_device_eval_batch_size=CFG.batch_size,
            num_train_epochs=CFG.num_train_epochs,
            weight_decay=CFG.weight_decay,
            dataloader_drop_last=True,
            dataloader_num_workers=2,
            fp16=CFG.use_fp16, # use FP16 for training, should increase training speed, but slightly decrease model accuracy
            
            # model evaludation and selection
            save_strategy='steps', # "steps",
            evaluation_strategy='steps', # "steps",
            eval_steps=CFG.save_steps,
            save_steps=CFG.save_steps,
            logging_steps=100,
            metric_for_best_model="mcrmse",
            greater_is_better=False,
            save_total_limit=1,
            load_best_model_at_end=True, # select best model
            report_to=CFG.report_to,
            run_name=f'{CFG.run_name}_fold{fold_num}' if CFG.run_name is not None else None
#             compile=True # if pytorch >2.0 for faster training
        )
        
        # Trainer is a high-level api to training, prediction loops
        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator = self.data_collator,
            train_dataset=train_dataset,
            eval_dataset=valid_dataset,
            tokenizer = preprocessor.tokenizer,
            compute_metrics = compute_mcrmse_numpy,
            optimizers=(optimizer, lr_scheduler)
        )

        trainer.train()
        # Finish logging current run
        wandb.finish()
        
    
    def predict(self, pretrained_model_path, test_dataset):
        
        model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_path)
        model.eval()
        
        # eg. "bert/fold_0/"
#         model_fold_dir = os.path.join(self.model_dir, str(fold)) 

        test_args = TrainingArguments(
            output_dir=CFG.model_name,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 2,   
            dataloader_drop_last = False,
        )

        # init trainer
        predict_trainer = Trainer(
            model = model, 
            args = test_args,
            tokenizer = preprocessor.tokenizer,
            data_collator = self.data_collator
        )

        predictions = predict_trainer.predict(test_dataset)[0]
        print(predictions)
        return predictions[:, 0], predictions[:, 1]
    

## Training/Prediction

In [ ]:
# Finetune a model 
if not CFG.predict_mode:
    cv = GroupKFold(n_splits=4)
    groups = train_df['prompt_id']
    for index, (train_idx, test_idx) in enumerate(cv.split(train_df['text'], train_df[['content', 'wording']], groups=groups), start=1):

        print(f'##### Starting Fold [{index}/{4}] #####')

        # Split into train and validation
        train_split, val_split = train_df[['text', 'content', 'wording']].iloc[train_idx], \
            train_df[['text', 'content', 'wording']].iloc[test_idx]

        # Create datasets and dataloaders
        train_dataset = preprocessor.run(train_split, train_mode=True)
        valid_dataset = preprocessor.run(val_split, train_mode=True)

        # Create model and perform training and validation
        model = DebertaRegressorModel()
        model.train(index, train_dataset, valid_dataset)
        
# Predict test dataset and submit results
else:
    test_dataset = preprocessor.run(test_df, train_mode=False)
    
    model = DebertaRegressorModel()

    test_df['content'], test_df['wording'] = model.predict(CFG.pretrained_model_path, test_dataset)

    test_df[['student_id', 'content', 'wording']].to_csv('submission.csv',index=False)
    display(pd.read_csv('submission.csv'))

## Wandb Results Report

In [17]:
if not CFG.predict_mode:
    %wandb yevhen-herasimov/nlp_course_deberta_finetuning/reports/Homework-5-Conclusion--Vmlldzo1ODEwMTc4